In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks= [
    StreamingStdOutCallbackHandler()
  ],
)

memory = ConversationBufferMemory(
    llm = chat,
    return_messages=True,
)

examples = [
  {
    "movie": "Harry Potter",
    "answer" : "🧙🏻‍♀️🪄👹",
  },
  {
    "movie":"Walking Dead",
    "answer": "🧟‍♀️🔫🍖",
  },
  {
    "movie":"Terminator",
    "answer" : "🤖💪🏽🚀"
  }
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "{movie}"),
  ("ai", "{answer}")
])

final_example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a movie expert. You know everything about the movie. If User give a movie title, you can answer using 3 emoji icons. If user ask question, please ask the question normally"),
  final_example_prompt,
  ("system", "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below."),
  MessagesPlaceholder(variable_name="history"),
  ("human","{movie}")
])

def load_memory(_):
    return memory.load_memory_variables({})['history']

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})
    print(result)

invoke_chain("Squid game")
invoke_chain("Back to the future")



🦑💰🎮content='🦑💰🎮'
🕰️🚗🔥content='🕰️🚗🔥'


In [10]:
invoke_chain("What was the first movie title i asked?")

You asked about "Squid Game."content='You asked about "Squid Game."'
